<a href="https://colab.research.google.com/github/bob8dod/NLP_SelfStudying/blob/main/Subword%20Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**서브워드 토크나이저(Subword Tokenizer)**
- 단어 토큰화에서 더 나아가 단어를 서브워드 단위까지 나누는 서브워드 토큰화
- OOV나 희귀 단어, 신조어와 같은 문제를 완화시킬 수 있음
- 하나의 단어를 여러 서브워드로 분리해서 단어를 인코딩 및 임베딩

# 센텐스피스(SentencePiece)

BPE 알고리즘과 Unigram Language Model Tokenizer를 구현한 센텐스피스  
사전 토큰화 작업없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않음

In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.2 MB/s 


## IMDB 리뷰 토큰화

In [ ]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f25d4a61ad0>)

In [ ]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review'].head()

0    My family and I normally do not watch local mo...
1    Believe it or not, this was at one time the wo...
2    After some internet surfing, I found the "Home...
3    One of the most unheralded great works of anim...
4    It was the Sixties, and anyone with long hair ...
Name: review, dtype: object

In [ ]:
print('리뷰 개수 :',len(train_df))

리뷰 개수 : 50000


센텐스피스의 입력으로 사용하기 위해서 데이터프레임을 txt 파일로 저장

In [ ]:
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

센텐스피스로 단어 집합과 각 단어에 고유한 정수를 부여

In [ ]:
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

- input : 학습시킬 파일
- model_prefix : 만들어질 모델 이름
- vocab_size : 단어 집합의 크기
- model_type : 사용할 모델 (unigram(default), bpe, char, word)
- max_sentence_length: 문장의 최대 길이
- pad_id, pad_piece: pad token id, 값
- unk_id, unk_piece: unknown token id, 값
- bos_id, bos_piece: begin of sentence token id, 값
- eos_id, eos_piece: end of sequence token id, 값
- user_defined_symbols: 사용자 정의 토큰

vocab 파일에서 학습된 서브워드들을 확인

In [ ]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
2300,▁points,-2297
3315,▁investig,-3312
4100,herent,-4097
4836,▁Cent,-4833
3198,▁Fran,-3195
2418,▁crime,-2415
553,▁There,-550
1853,▁shock,-1850
3016,elly,-3013
4175,▁repeated,-4172


model 파일을 로드하여 단어 시퀀스를 정수 시퀀스로 바꾸는 인코딩 작업이나 반대로 변환하는 디코딩 작업

In [ ]:
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

In [ ]:
# 모델 테스트
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



In [ ]:
# 정수 시퀀스로부터 문장으로 변환 _ 디코딩
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'I have waited a long time for someone to film'

In [ ]:
# encode : 문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환 가능
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


## 네이버 영화 리뷰 토큰화

In [ ]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
naver_df = pd.read_table('ratings.txt')
print('리뷰 개수 :',len(naver_df))
naver_df.head(3)

리뷰 개수 : 200000


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1


In [ ]:
# 전처리 과정
print(naver_df.isnull().values.any())
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

True
False
리뷰 개수 : 199992


센텐스피스의 입력으로 사용하기 위해서 데이터프레임을 txt 파일로 저장

In [ ]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

- input : 학습시킬 파일
- model_prefix : 만들어질 모델 이름
- vocab_size : 단어 집합의 크기
- model_type : 사용할 모델 (unigram(default), bpe, char, word)
- max_sentence_length: 문장의 최대 길이
- pad_id, pad_piece: pad token id, 값
- unk_id, unk_piece: unknown token id, 값
- bos_id, bos_piece: begin of sentence token id, 값
- eos_id, eos_piece: end of sequence token id, 값
- user_defined_symbols: 사용자 정의 토큰  

**vocab 생성이 완료되면 `naver.model, naver.vocab` 파일 두개가 생성**  
- .vocab 에서 학습된 subwords를 확인  
- model 파일을 로드하여 단어 시퀀스를 정수 시퀀스로 바꾸는 인코딩 작업이나 반대로 변환하는 디코딩 작업

In [ ]:
# naver.vocab 에서 학습된 subwords를 확인
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
len(vocab_list) #설정한대로 5000개의 서브워드가 단어 집합에 존재
vocab_list.sample(10) # 샘플 출력

,0,1
1608,▁탄탄,-1605
1069,▁옛날,-1066
652,내용,-649
2629,▁미스,-2626
4093,랫,-4090
2347,성애,-2344
2617,디어,-2614
2642,▁채널,-2639
336,▁마음,-333
2260,▁삶을,-2257


In [ ]:
# model 파일을 로드하여 단어 시퀀스를 정수 시퀀스로 바꾸는 인코딩 작업이나 반대로 변환하는 디코딩 작업
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [ ]:
#모델 평가
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line)) # 서브 토큰화
  print(sp.encode_as_ids(line)) # 정수 인덱싱
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



In [ ]:
print('단어 집합의 크기: ',sp.GetPieceSize())
print('정수로부터 맵핑되는 서브 워드로 변환:\t[정수: 430] => ', sp.IdToPiece(430))
print('서브워드로부터 맵핑되는 정수로 변환: \t[워드:"스럽"]=> ',sp.PieceToId('스럽'))
print('정수 시퀀스로부터 문장으로 변환: ', sp.DecodeIds([54, 200, 821, 85]))
print('서브워드 시퀀스로부터 문장으로 변환: ', sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']))
print()
print('문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환: ',)
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

단어 집합의 크기:  5000
정수로부터 맵핑되는 서브 워드로 변환:	[정수: 430] =>  스럽
서브워드로부터 맵핑되는 정수로 변환: 	[워드:"스럽"]=>  430
정수 시퀀스로부터 문장으로 변환:  진짜 최고의 영화입니다 ᄏᄏ
서브워드 시퀀스로부터 문장으로 변환:  진짜 최고의 영화입니다 ᄏᄏ

문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환: 
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]


# Word Piece Model(WPM)

- 텐서플로우를 통해 사용할 수 있는 서브워드 토크나이저
- BPE와 유사한 알고리즘
- 패키지를 통해 쉽게 단어들을 서브워드들로 분리할 수 있음

## 네이버 영화 리뷰 토큰화

In [ ]:
import tensorflow_datasets as tfds
import urllib.request

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')

In [ ]:
#전처리1
print(train_data.isnull().sum())
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

id          0
document    5
label       0
dtype: int64
False


tfds.features.text.SubwordTextEncoder.build_from_corpus의 인자로 네이버 영화 리뷰 데이터를 넣어서, 서브워드들로 이루어진 단어 집합(Vocabulary)를 생성하고, 각 서브워드에 고유한 정수를 부여

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    train_data['document'], target_vocab_size=2**13)
print(tokenizer.subwords[:100]) # 토큰화 된 100개의 서브워드들을 출력

['. ', '..', '영화', '이_', '...', '의_', '는_', '도_', '다', ', ', '을_', '고_', '은_', '가_', '에_', '.. ', '한_', '너무_', '정말_', '를_', '고', '게_', '영화_', '지', '... ', '진짜_', '이', '다_', '요', '만_', '? ', '과_', '나', '가', '서_', '지_', '로_', '으로_', '아', '어', '....', '음', '한', '수_', '와_', '도', '네', '그냥_', '나_', '더_', '왜_', '이런_', '면_', '기', '하고_', '보고_', '하는_', '서', '좀_', '리', '자', '스', '안', '! ', '에서_', '영화를_', '미', 'ㅋㅋ', '네요', '시', '주', '라', '는', '오', '없는_', '에', '해', '사', '!!', '영화는_', '마', '잘_', '수', '영화가_', '만', '본_', '로', '그_', '지만_', '대', '은', '비', '의', '일', '개', '있는_', '없다', '함', '구', '하']


encode()를 통해 임의로 선택한 21번째 샘플을 인코딩

In [ ]:
print('Tokenized sample question: {}'.format(tokenizer.encode(train_data['document'][20])))

Tokenized sample question: [669, 4700, 17, 1749, 8, 96, 131, 1, 48, 2239, 4, 7466, 32, 1274, 2655, 7, 80, 749, 1254]


decode()를 통해서 다시 역으로 디코딩

In [ ]:
sample_string = train_data['document'][21]

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [570, 892, 36, 584, 159, 7091, 201]
기존 문장: 보면서 웃지 않는 건 불가능하다


In [ ]:
# 임의의 문장으로 테스트 해보기
sample_string = '이 영화 굉장히 재밌다 킄핫핫ㅎ'

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [4, 23, 1364, 2157, 8235, 8128, 8130, 8235, 8147, 8169, 8235, 8147, 8169, 393]
기존 문장: 이 영화 굉장히 재밌다 킄핫핫ㅎ


기존 훈련 데이터에 없을만한 '킄핫핫'  
서브워드텍스트인코더는 이 경우 음절 이하 단위로 분리하고, 또한 정상적으로 디코딩